<a href="https://colab.research.google.com/github/MentalCoder91/test/blob/main/GoogleNEt_Excercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Excercise**
(Connect to T4 GPU)

Implement GoogleNet architecture for image classification on the CIFAR-10 dataset.


In [7]:
#import the Libraries
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Concatenate, Dropout, Flatten

The CIFAR-10 dataset is a widely used benchmark dataset in the field of computer vision. It consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class. The classes are: airplane, automobile, bird, cat, deer, dog, frog, horse, ship, and truck. It's often used for tasks such as object recognition, image classification, and machine learning algorithm evaluation.

Link : https://www.cs.toronto.edu/~kriz/cifar.html

In [15]:
# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()


x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Convert class vectors to binary class matrices (one-hot encoding)
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)





In [21]:
# Define the Inception module
def inception_module(x, filters):
    # 1x1 convolutional layer with [0] filters, kernel size (1x1), padding, and ReLU activation
     conv1x1 = Conv2D(filters=filters[0], kernel_size=(1, 1), padding='same', activation='relu')(x)
    # 3x3 convolutional layer with [1] filters, kernel size (3x3), padding, and ReLU activation
     conv3x3 = Conv2D(filters=filters[1], kernel_size=(3, 3), padding='same', activation='relu')(x)
    # 5x5 convolutional layer [2] filters, kernel size (5x5), padding, and ReLU activation
     conv5x5 = Conv2D(filters=filters[2], kernel_size=(5, 5), padding='same', activation='relu')(x)
    # Max pooling layer with specified pool size (3x3), strides (1x1), and padding
     max_pool_conv = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)
    # 1x1 convolutional layer applied after max pooling, with [3] filters, kernel size (1x1), padding, and ReLU activation
     conv1x11 = Conv2D(filters=filters[3], kernel_size=(1, 1), padding='same', activation='relu')(x)
    # Concatenate the outputs of all convolutional operations along the last axis
     output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, max_pool_conv])
     return output

# Define the GoogleNet architecture
def google_net(input_shape, num_classes):
    # Input layer with specified input shape
    inputs = Input(shape=input_shape)
    # Initial convolutional layer with 64 filters, kernel size (7x7), strides (2x2), padding, and ReLU activation
    x = Conv2D(filters=64, kernel_size=(7, 7), strides=(2, 2), padding='same', activation='relu')(inputs)
    # Max pooling layer with pool size (3x3), strides (2x2), and padding
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    # First Inception module with [64, 128, 32, 32] filters for each convolutional operation
    x = inception_module(x, filters=[64, 128, 32, 32])
    # Second Inception module with [128, 192, 96, 64] filters for each convolutional operation
    x = inception_module(x, filters=[128, 192, 96, 64])
    # Max pooling layer with pool size (3x3), strides (2x2), and padding
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    # Third Inception module with [192, 208, 48, 64] filters for each convolutional operation
    x = inception_module(x, filters=[192, 208, 48, 64])
    # Fourth Inception module with [160, 224, 64, 64] filters for each convolutional operation
    x = inception_module(x, filters=[160, 224, 64, 64])
    # Global average pooling layer
    x = GlobalAveragePooling2D()(x)
    # Dropout layer with dropout rate of 0.4
    x = Dropout(0.4)(x)
    # Output layer with specified number of classes and softmax activation
    outputs = Dense(num_classes, activation='softmax')(x)
    # Create the model with specified inputs and outputs
    model = tf.keras.Model(inputs, outputs)
    return model


In [22]:
input_shape = x_train.shape[1:]
input_shape

(32, 32, 3)

In [23]:
# Create and compile the model
input_shape = x_train.shape[1:]
num_classes = 10
model = google_net(input_shape, num_classes)
#Compile the model with optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [24]:
# Train the model with epochs=5, steps_per_epoch = 800,batch_size=32
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), steps_per_epoch=800)


Epoch 1/5
800/800 [==============================] - 29s 28ms/step - loss: 1.7864 - accuracy: 0.3254 - val_loss: 1.4807 - val_accuracy: 0.4502
Epoch 2/5
800/800 [==============================] - 22s 27ms/step - loss: 1.4147 - accuracy: 0.4791 - val_loss: 1.2757 - val_accuracy: 0.5317
Epoch 3/5
800/800 [==============================] - 23s 29ms/step - loss: 1.2555 - accuracy: 0.5464 - val_loss: 1.1944 - val_accuracy: 0.5736
Epoch 4/5
800/800 [==============================] - 24s 29ms/step - loss: 1.1698 - accuracy: 0.5802 - val_loss: 1.1034 - val_accuracy: 0.6096
Epoch 5/5
800/800 [==============================] - 24s 29ms/step - loss: 1.0718 - accuracy: 0.6157 - val_loss: 1.0120 - val_accuracy: 0.6476


In [25]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)

313/313 [==============================] - 3s 10ms/step - loss: 1.0120 - accuracy: 0.6476
Test accuracy: 0.647599995136261
Test loss: 1.0120049715042114


### "Wishing you all the best on your learning journey! May each challenge be a stepping stone to your success. Happy coding!"